## basic test

In [41]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [42]:
#-- REAL WORLD configuration, which is attached to the physics engine
# accessing this directly would be cheating!
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

In [43]:
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [44]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/pandasTable.g")

In [45]:
#-- using the viewer, you can view configurations or paths
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [46]:
#-- the following is the simulation loop
tau = .01

for t in range(5):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    [y,J] = C.evalFeature(ry.FS.position, ["R_gripper"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ [0.,0.,-1e-1];

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

## doing things relative to an object

In [47]:
# add a new frame to the MODEL configuration
# (Perception will later have to do exactly this: add perceived objects to the model)
capsule = C.addFrame("capsule")
box     = C.addFrame("box")

In [48]:
# set frame parameters, associate a shape to the frame, 
# capsule.setPosition([.8,0,1.5])
# capsule.setQuaternion([1,0,.5,0])

capsule.setPosition([0.3,0,0.8])
capsule.setQuaternion([1,0,0,0])

capsule.setShape(ry.ST.capsule, [.2,.02])
capsule.setColor([1,0,1])


box.setPosition([-.8,0,1.5])
box.setQuaternion([1,0,.5,0])
box.setShape(ry.ST.box, [0.1,0.1,0.1])
box.setColor([1,1,0])


V.setConfiguration(C)
#V.recopyMeshes(C) #this is rarely necessary, only when you change meshes within C

In [49]:
#-- the following is the simulation loop
tau = .01

#[y_itself ,J_itself]  = C.evalFeature(ry.FS.position, ["L_gripperCenter"])
for t in range(500):
    y = []
    J = []
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    
    [y_capsule,J_capsule] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "capsule"])
    [y_box    ,J_box]     = C.evalFeature(ry.FS.positionDiff, ["L_gripperCenter", "box"])
    [y_LGrip  ,J_LGrip]   = C.evalFeature(ry.FS.position, ["L_gripperCenter"])

    [y_capZZ, J_capsuleZZ] = C.evalFeature(ry.FS.scalarProductZZ, ["R_gripperCenter", "capsule"])
    [y_capXZ, J_capsuleXZ] = C.evalFeature(ry.FS.scalarProductZZ, ["R_gripperCenter", "capsule"])
    
    # merge all tasks
    
    # reach capsule
    y = y_capsule/tau
    J = J_capsule/tau
    
    # stay close to initial position
    #y  = np.concatenate((y ,  (y_LGrip - y_itself)))
    #J = np.vstack((J,  J_LGrip))
    
    
    
    
    # reach box
    y  = np.concatenate((y , y_capZZ/tau, y_capXZ/tau, y_box/tau))
    J = np.vstack((J,  J_capsuleZZ/tau, J_capsuleXZ/tau, J_box/tau))
    
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ (-y);
    
    # stop Left gripper lifting
    vel[0] = 0
    
    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

How could you align the gripper for a proper grasp?

In [40]:
S=0
V=0
C=0
RealWorld=0

In [24]:
vel

array([ 0.00000000e+00, -1.26776098e-03,  8.59466957e-03,  5.92841712e-04,
        3.02174005e-02,  7.70084399e-04,  2.08040735e-02, -2.32976877e-08,
        0.00000000e+00, -9.93030449e-04,  1.08702300e-02, -3.96479269e-03,
        7.57881804e-03, -5.53363185e-04,  7.43794935e-03, -6.42306208e-09,
        0.00000000e+00])

In [10]:
y

array([ 1.06328881e-03, -3.31015539e-03,  9.70742041e-03,  2.65724035e-01,
       -8.28154117e-02, -8.14342469e+00,  3.87184783e-01])